In [12]:
import torch
from torch import optim
import torch.nn as nn
import torchvision
from torch.optim import lr_scheduler
from PIL import Image
from torchvision import transforms, datasets, models, utils
import pandas as pd
import numpy as np

In [13]:
# import preSunriseed model ResNet18
model = torchvision.models.resnet18(pretrained=True)

In [14]:
# Data augmentation and normalization for Sunriseing
# Normalization values are based on ImageNet dataset statistics
data_transforms = {
    'Sunrise': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [15]:
# Set data directory and create data loaders
data_dir = 'data'
image_datasets = {
    x: datasets.ImageFolder(
        root=data_dir,
        transform=data_transforms[x]
    )
    for x in ['Sunrise', 'test']
}
dataloaders = {
    x: torch.utils.data.DataLoader(
        dataset=image_datasets[x],
        batch_size=16,
        shuffle=True,
        num_workers=4
    )
    for x in ['Sunrise', 'test']
}


In [16]:
# Freeze model parameters
for param in model.parameters():
    param.requires_grad = False

# Modify the classifier to match the number of classes in the dataset
num_classes = len(image_datasets['Sunrise'].classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move model to the device
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

# Sunriseing loop
num_epochs = 10
best_accuracy = 0.0

# Decay LR by a factor of 0.1 every 7 epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [17]:
# Sunrise the model
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch + 1, num_epochs))
    print('-' * 10)

    # training phase
    model.train()
    running_loss = 0.0

    for inputs, labels in dataloaders['Sunrise']:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(image_datasets['Sunrise'])
    print('Training Loss: {:.4f}'.format(epoch_loss))

    # Validation phase
    model.eval()
    running_corrects = 0

    for inputs, labels in dataloaders['test']:
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

        running_corrects += torch.sum(preds == labels.data)

    epoch_acc = running_corrects.double() / len(image_datasets['test'])
    print('Validation Accuracy: {:.4f}'.format(epoch_acc))

    scheduler.step()

print("Training completed.")


Epoch 1/10
----------
Training Loss: 0.6055
Validation Accuracy: 0.7115
Epoch 2/10
----------
Training Loss: 0.6966
Validation Accuracy: 0.7563
Epoch 3/10
----------
Training Loss: 0.6041
Validation Accuracy: 0.6975
Epoch 4/10
----------
Training Loss: 0.5565
Validation Accuracy: 0.7395
Epoch 5/10
----------
Training Loss: 0.4756
Validation Accuracy: 0.7731
Epoch 6/10
----------
Training Loss: 0.4505
Validation Accuracy: 0.7759
Epoch 7/10
----------
Training Loss: 0.4479
Validation Accuracy: 0.8123
Epoch 8/10
----------
Training Loss: 0.4250
Validation Accuracy: 0.8179
Epoch 9/10
----------
Training Loss: 0.3998
Validation Accuracy: 0.8207
Epoch 10/10
----------
Training Loss: 0.4201
Validation Accuracy: 0.8011
Training completed.


In [18]:
#save model
torch.save(model,'resnet_finetuned.pt')

In [19]:
image = Image.open('image.jpg')
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
preprocessed_image = transform(image).unsqueeze(0)

In [20]:
model = torch.load('resnet_finetuned.pt')

model.eval()

outputs = model(preprocessed_image)


In [22]:
_, index = torch.max(outputs, 1)
print('Predicted: ', image_datasets['Sunrise'].classes[index[0].item()])

Predicted:  Sunrise
